In [131]:
from lsst.daf.butler import Butler
import numpy as np
from lsst.meas.base import ForcedMeasurementTask, ForcedMeasurementConfig
import lsst.geom as geom
import lsst.afw.table as afwTable
from lsst.summit.utils import ConsDbClient

In [195]:
butler = Butler("embargo",collections='u/rea3/too_0725J/SN10//20250814T211435Z')


import os 
del os.environ["http_proxy"]



KeyError: 'http_proxy'

In [196]:
ConsClient = ConsDbClient('http://consdb-pq.consdb:8080/consdb')

In [3]:
# name ra dec
# AT2025shz   231.072434 -37.622634
# AT2025sib   235.548892 -35.002876
# AT2025sia   234.538140 -36.254760
# AT2025srk  234.755524  -36.425993
# AT2025srl  234.90473   -36.726032
# AT2025srm  232.987656  -36.816564

names = ['AT2025shz', 'AT2025sib', 'AT2025sia', 'AT2025srk', 'AT2025srl', 'AT2025srm']
ras = np.array([231.072434, 235.548892, 234.538140, 234.755524, 234.90473, 232.987656])
decs = np.array([-37.622634, -35.002876, -36.254760, -36.425993, -36.726032, -36.816564])
ids = np.arange(0,6)


In [4]:
schema = afwTable.SourceTable.makeMinimalSchema()
table = afwTable.SourceTable.make(schema)

forced_config = ForcedMeasurementConfig()

forced_config.plugins.names = [
    "base_PixelFlags",
    "base_TransformedCentroidFromCoord",
    "base_PsfFlux",
    "base_LocalBackground",                     
]
forced_config.slots.shape = None
forced_config.copyColumns = {'id': 'diaObjectId', 'coord_ra': 'coord_ra', 'coord_dec': 'coord_dec'}
forced_config.slots.centroid = "base_TransformedCentroidFromCoord"
forced_config.slots.psfFlux = "base_PsfFlux"
forced_config.doReplaceWithNoise = False

forced_task = ForcedMeasurementTask(config=forced_config, refSchema=schema)

In [6]:
explist = [2025072800100, ]

In [5]:
ExpsOfInterest = """instrument exposure detector
LSSTCam 2025072800105 25
LSSTCam 2025072800115 25
LSSTCam 2025072800125 25
LSSTCam 2025072800135 25
LSSTCam 2025072800145 25
LSSTCam 2025072800155 25
LSSTCam 2025072800104 170
LSSTCam 2025072800114 170
LSSTCam 2025072800124 170
LSSTCam 2025072800134 170
LSSTCam 2025072800144 170
LSSTCam 2025072800154 170
LSSTCam 2025072800105 21
LSSTCam 2025072800115 21
LSSTCam 2025072800125 21
LSSTCam 2025072800135 21
LSSTCam 2025072800145 21
LSSTCam 2025072800155 21
LSSTCam 2025072800104 167
LSSTCam 2025072800114 167
LSSTCam 2025072800124 167
LSSTCam 2025072800134 167
LSSTCam 2025072800144 167
LSSTCam 2025072800154 167
LSSTCam 2025072800104 185
LSSTCam 2025072800114 185
LSSTCam 2025072800124 185
LSSTCam 2025072800134 185
LSSTCam 2025072800144 185
LSSTCam 2025072800154 185
LSSTCam 2025072800100 131
LSSTCam 2025072800110 131
LSSTCam 2025072800120 131
LSSTCam 2025072800130 131
LSSTCam 2025072800140 131
LSSTCam 2025072800150 131
LSSTCam 2025072800105 97
LSSTCam 2025072800115 97
LSSTCam 2025072800125 97
LSSTCam 2025072800135 97
LSSTCam 2025072800145 97
LSSTCam 2025072800155 97
LSSTCam 2025072800105 56
LSSTCam 2025072800115 56
LSSTCam 2025072800125 56
LSSTCam 2025072800135 56
LSSTCam 2025072800145 56
LSSTCam 2025072800155 56"""

In [13]:
ToOExps = np.array([x.split(" ") for x in ExpsOfInterest.split("\n")][1:])

In [111]:
zp = 

for e in ToOExps:
    exp = int(e[1])
    det = int(e[2])
    try:
        diff_refs = list(butler.query_datasets(
          "difference_image",
          where=f"exposure={exp} and instrument='LSSTCam' and detector={det}"
        ))
    except:
        continue
    
    bbox = butler.get(
        "difference_image.bbox",
        dataId=diff_refs[0].dataId
    )
    wcs = butler.get(
        "difference_image.wcs",
        dataId=diff_refs[0].dataId
    )                    

    found = np.array([False]*len(ras))
    for i,(ra,dec) in enumerate(zip(ras,decs)):
        if bbox.contains(geom.Point2I(wcs.skyToPixel(geom.SpherePoint(ra,dec,geom.degrees)))):
            found[i] = True
    n_src = np.sum(found==True)
    n_mask = found == True
    if n_src == 0:
        continue
    print('Found',exp, det, n_src, 'sources')
    diff = butler.get('difference_image', dataId=diff_refs[0].dataId)
    temp = butler.get('template_matched', dataId=diff_refs[0].dataId)
    sci = butler.get('preliminary_visit_image', dataId=diff_refs[0].dataId)

    if (e==ToOExps[0]).all(): # If this is the first forced source measurement, instantiate the master catalog
        tempMasterCat = afwTable.SourceCatalog(forced_task.generateMeasCat(temp, ref_catalog, 
                                                                           temp.getWcs()).getSchema()).asAstropy()
        sciMasterCat = afwTable.SourceCatalog(forced_task.generateMeasCat(sci, ref_catalog, 
                                                                          sci.getWcs()).getSchema()).asAstropy()
        diffMasterCat = afwTable.SourceCatalog(forced_task.generateMeasCat(diff, ref_catalog, 
                                                                           diff.getWcs()).getSchema()).asAstropy()

    schema = afwTable.SourceTable.makeMinimalSchema()
    ref_catalog = afwTable.SourceCatalog(schema)
    ref_catalog.reserve(n_src)

    for object_id, ra, dec in zip(ids[n_mask],ras[n_mask],decs[n_mask]):
        record = ref_catalog.addNew()
        record.setId(object_id)
        record.setCoord(geom.SpherePoint(ra, dec, geom.degrees))

    # Generate forced photometry catalogs
    forced_sci_cat = forced_task.generateMeasCat(sci, ref_catalog, sci.getWcs())
    forced_diff_cat = forced_task.generateMeasCat(diff, ref_catalog, diff.getWcs())
    forced_temp_cat = forced_task.generateMeasCat(temp, ref_catalog, temp.getWcs())

    # Attach footprints to the catalogs - just use PSF
    forced_task.attachPsfShapeFootprints(forced_sci_cat, sci)
    forced_task.attachPsfShapeFootprints(forced_diff_cat, diff)
    # Use psf size of the difference image, because template PSF is somewhat arbitrary
    forced_task.attachPsfShapeFootprints(forced_temp_cat, diff)

    forced_task.run(forced_sci_cat, sci, ref_catalog, sci.getWcs())
    forced_task.run(forced_diff_cat, diff, ref_catalog, diff.getWcs())
    forced_task.run(forced_temp_cat, temp, ref_catalog, temp.getWcs())

    # Append results to the master catalog
    tempMasterCat.add_row(forced_temp_cat.asAstropy()[-1])
    sciMasterCat.add_row(forced_sci_cat.asAstropy()[-1])
    diffMasterCat.add_row(forced_diff_cat.asAstropy()[-1])

Found 2025072800105 25 1 sources


Found 2025072800115 25 1 sources


Found 2025072800135 25 1 sources


Found 2025072800145 25 1 sources


Found 2025072800104 170 1 sources


...

Found 2025072800145 97 1 sources


Found 2025072800105 56 1 sources


Found 2025072800115 56 1 sources


Found 2025072800135 56 1 sources


Found 2025072800145 56 1 sources


In [121]:
tempMasterCat

id,coord_ra,coord_dec,parent,diaObjectId,base_TransformedCentroidFromCoord_x,slot_Centroid_x,base_TransformedCentroidFromCoord_y,slot_Centroid_y,base_LocalBackground_instFlux,base_LocalBackground_instFluxErr,base_LocalBackground_flag,base_LocalBackground_flag_noGoodPixels,base_LocalBackground_flag_noPsf,base_PixelFlags_flag,base_PixelFlags_flag_offimage,base_PixelFlags_flag_edge,base_PixelFlags_flag_nodata,base_PixelFlags_flag_interpolated,base_PixelFlags_flag_saturated,base_PixelFlags_flag_cr,base_PixelFlags_flag_bad,base_PixelFlags_flag_suspect,base_PixelFlags_flag_edgeCenter,base_PixelFlags_flag_nodataCenter,base_PixelFlags_flag_interpolatedCenter,base_PixelFlags_flag_saturatedCenter,base_PixelFlags_flag_crCenter,base_PixelFlags_flag_badCenter,base_PixelFlags_flag_suspectCenter,base_PixelFlags_flag_edgeCenterAll,base_PixelFlags_flag_nodataCenterAll,base_PixelFlags_flag_interpolatedCenterAll,base_PixelFlags_flag_saturatedCenterAll,base_PixelFlags_flag_crCenterAll,base_PixelFlags_flag_badCenterAll,base_PixelFlags_flag_suspectCenterAll,base_PsfFlux_instFlux,slot_PsfFlux_instFlux,base_PsfFlux_instFluxErr,slot_PsfFlux_instFluxErr,base_PsfFlux_area,slot_PsfFlux_area,base_PsfFlux_chi2,slot_PsfFlux_chi2,base_PsfFlux_npixels,slot_PsfFlux_npixels,base_PsfFlux_flag,slot_PsfFlux_flag,base_PsfFlux_flag_noGoodPixels,slot_PsfFlux_flag_noGoodPixels,base_PsfFlux_flag_edge,slot_PsfFlux_flag_edge,base_InvalidPsf_flag
,rad,rad,,,pix,pix,pix,pix,ct,ct,,,,,,,,,,,,,,,,,,,,,,,,,,,ct,ct,ct,ct,pix,pix,,,pix,pix,,,,,,,
int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,float64,float64,float64,float64,float32,float32,float32,float32,int32,int32,bool,bool,bool,bool,bool,bool,bool
1,4.097256831044569,-0.6357535111584068,0,3,1954.2790343613942,1954.2790343613942,1386.270225048458,1386.270225048458,1.5208662957925858,13.944364719677093,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2038.3265254079067,2038.3265254079067,1.320255018895855,1.320255018895855,92.4373,92.4373,788475970.0,788475970.0,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,1981.1801828135558,1981.1801828135558,1379.0027781446515,1379.0027781446515,-0.7262133918580695,5.984449936502553,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2006.0060454491168,2006.0060454491168,6.989040834242434,6.989040834242434,111.35246,111.35246,17590726.0,17590726.0,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,1951.3677341746006,1951.3677341746006,1374.0580573520383,1374.0580573520383,0.5844207552719164,8.735313806016697,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2318.023018221967,2318.023018221967,1.1813367963339205,1.1813367963339205,115.87338,115.87338,775280600.0,775280600.0,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,1982.0077220867533,1982.0077220867533,1374.2101014790678,1374.2101014790678,-0.11164845717230765,10.016931193779314,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1831.9369049284655,1831.9369049284655,16.13234245693026,16.13234245693026,88.86893,88.86893,4060119.5,4060119.5,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,2304.8353155439017,2304.8353155439017,3260.8390207450793,3260.8390207450793,2.2026372179796225,17.161832476860642,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [128]:
sciMasterCat

id,coord_ra,coord_dec,parent,diaObjectId,base_TransformedCentroidFromCoord_x,slot_Centroid_x,base_TransformedCentroidFromCoord_y,slot_Centroid_y,base_LocalBackground_instFlux,base_LocalBackground_instFluxErr,base_LocalBackground_flag,base_LocalBackground_flag_noGoodPixels,base_LocalBackground_flag_noPsf,base_PixelFlags_flag,base_PixelFlags_flag_offimage,base_PixelFlags_flag_edge,base_PixelFlags_flag_nodata,base_PixelFlags_flag_interpolated,base_PixelFlags_flag_saturated,base_PixelFlags_flag_cr,base_PixelFlags_flag_bad,base_PixelFlags_flag_suspect,base_PixelFlags_flag_edgeCenter,base_PixelFlags_flag_nodataCenter,base_PixelFlags_flag_interpolatedCenter,base_PixelFlags_flag_saturatedCenter,base_PixelFlags_flag_crCenter,base_PixelFlags_flag_badCenter,base_PixelFlags_flag_suspectCenter,base_PixelFlags_flag_edgeCenterAll,base_PixelFlags_flag_nodataCenterAll,base_PixelFlags_flag_interpolatedCenterAll,base_PixelFlags_flag_saturatedCenterAll,base_PixelFlags_flag_crCenterAll,base_PixelFlags_flag_badCenterAll,base_PixelFlags_flag_suspectCenterAll,base_PsfFlux_instFlux,slot_PsfFlux_instFlux,base_PsfFlux_instFluxErr,slot_PsfFlux_instFluxErr,base_PsfFlux_area,slot_PsfFlux_area,base_PsfFlux_chi2,slot_PsfFlux_chi2,base_PsfFlux_npixels,slot_PsfFlux_npixels,base_PsfFlux_flag,slot_PsfFlux_flag,base_PsfFlux_flag_noGoodPixels,slot_PsfFlux_flag_noGoodPixels,base_PsfFlux_flag_edge,slot_PsfFlux_flag_edge,base_InvalidPsf_flag
,rad,rad,,,pix,pix,pix,pix,ct,ct,,,,,,,,,,,,,,,,,,,,,,,,,,,ct,ct,ct,ct,pix,pix,,,pix,pix,,,,,,,
int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,float64,float64,float64,float64,float32,float32,float32,float32,int32,int32,bool,bool,bool,bool,bool,bool,bool
1,4.097256831044569,-0.6357535111584068,0,3,1954.2790343613942,1954.2790343613942,1386.270225048458,1386.270225048458,10.011682105096487,45.374083329354804,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,19086.73829007853,19086.73829007853,417.7134551668645,417.7134551668645,92.4373,92.4373,10322.717,10322.717,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,1981.1801828135558,1981.1801828135558,1379.0027781446515,1379.0027781446515,5.506473801986576,39.48822264662526,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,23245.370347545093,23245.370347545093,436.8014258895065,436.8014258895065,111.35246,111.35246,6474.3887,6474.3887,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,1951.3677341746006,1951.3677341746006,1374.0580573520383,1374.0580573520383,6.268764979270592,44.91115996713921,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,16224.983842836293,16224.983842836293,459.5664042270487,459.5664042270487,115.87338,115.87338,7062.2456,7062.2456,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,1982.0077220867533,1982.0077220867533,1374.2101014790678,1374.2101014790678,7.324527872537826,39.09002519531311,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,26009.815463365583,26009.815463365583,380.7800985706804,380.7800985706804,88.86893,88.86893,9762.607,9762.607,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,2304.8353155439017,2304.8353155439017,3260.8390207450793,3260.8390207450793,8.519092801996132,50.13309365065347,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [115]:
diffMasterCat

id,coord_ra,coord_dec,parent,diaObjectId,base_TransformedCentroidFromCoord_x,slot_Centroid_x,base_TransformedCentroidFromCoord_y,slot_Centroid_y,base_LocalBackground_instFlux,base_LocalBackground_instFluxErr,base_LocalBackground_flag,base_LocalBackground_flag_noGoodPixels,base_LocalBackground_flag_noPsf,base_PixelFlags_flag,base_PixelFlags_flag_offimage,base_PixelFlags_flag_edge,base_PixelFlags_flag_nodata,base_PixelFlags_flag_interpolated,base_PixelFlags_flag_saturated,base_PixelFlags_flag_cr,base_PixelFlags_flag_bad,base_PixelFlags_flag_suspect,base_PixelFlags_flag_edgeCenter,base_PixelFlags_flag_nodataCenter,base_PixelFlags_flag_interpolatedCenter,base_PixelFlags_flag_saturatedCenter,base_PixelFlags_flag_crCenter,base_PixelFlags_flag_badCenter,base_PixelFlags_flag_suspectCenter,base_PixelFlags_flag_edgeCenterAll,base_PixelFlags_flag_nodataCenterAll,base_PixelFlags_flag_interpolatedCenterAll,base_PixelFlags_flag_saturatedCenterAll,base_PixelFlags_flag_crCenterAll,base_PixelFlags_flag_badCenterAll,base_PixelFlags_flag_suspectCenterAll,base_PsfFlux_instFlux,slot_PsfFlux_instFlux,base_PsfFlux_instFluxErr,slot_PsfFlux_instFluxErr,base_PsfFlux_area,slot_PsfFlux_area,base_PsfFlux_chi2,slot_PsfFlux_chi2,base_PsfFlux_npixels,slot_PsfFlux_npixels,base_PsfFlux_flag,slot_PsfFlux_flag,base_PsfFlux_flag_noGoodPixels,slot_PsfFlux_flag_noGoodPixels,base_PsfFlux_flag_edge,slot_PsfFlux_flag_edge,base_InvalidPsf_flag
,rad,rad,,,pix,pix,pix,pix,ct,ct,,,,,,,,,,,,,,,,,,,,,,,,,,,ct,ct,ct,ct,pix,pix,,,pix,pix,,,,,,,
int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,float64,float64,float64,float64,float32,float32,float32,float32,int32,int32,bool,bool,bool,bool,bool,bool,bool
1,4.097256831044569,-0.6357535111584068,0,3,1954.2790343613942,1954.2790343613942,1386.270225048458,1386.270225048458,1.6021729860207725,41.52457527538405,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,16824.76015888126,16824.76015888126,417.73641833011584,417.73641833011584,92.4373,92.4373,1900.5968,1900.5968,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,1981.1801828135558,1981.1801828135558,1379.0027781446515,1379.0027781446515,0.7495355429917515,36.98340010794397,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,21120.567118410334,21120.567118410334,437.2351868955715,437.2351868955715,111.35246,111.35246,1718.538,1718.538,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,1951.3677341746006,1951.3677341746006,1374.0580573520383,1374.0580573520383,0.7237430631409002,41.35527177650428,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,13764.075731001176,13764.075731001176,459.5826301313524,459.5826301313524,115.87338,115.87338,1865.0103,1865.0103,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,1982.0077220867533,1982.0077220867533,1374.2101014790678,1374.2101014790678,1.2360368294595283,35.23721564979787,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,24029.06313088761,24029.06313088761,381.3000635586286,381.3000635586286,88.86893,88.86893,1868.226,1868.226,1681,1681,False,False,False,False,False,False,False
1,4.097256831044569,-0.6357535111584068,0,3,2304.8353155439017,2304.8353155439017,3260.8390207450793,3260.8390207450793,-4.051997469157594,46.06611824245236,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

### Get zero points from consDB

In [192]:
qry = """SELECT cvq.zero_point, cex.detector,e.exposure_id 
FROM cdb_LSSTCam.ccdvisit1_quicklook as cvq, cdb_LSSTCam.exposure as e, cdb_LSSTCam.ccdexposure as cex
WHERE e.exposure_id={} and cex.detector={} and ;""".format(exp,det)

In [197]:
qry

'SELECT cvq.zero_point, cex.detector,e.exposure_id \nFROM cdb_LSSTCam.ccdvisit1_quicklook as cvq, cdb_LSSTCam.exposure as e, cdb_LSSTCam.ccdexposure as cex\nWHERE e.exposure_id=2025072800155 and cex.detector=56;'

In [198]:
ConsClient.query(qry)

KeyboardInterrupt: 

In [176]:
def fluxToMag(flux,zp):
    return -2.5*np.log10(flux) + zp